In [ ]:
# Mount Google Drive containing data
from google.colab import drive
drive.mount('/content/drive')
datadir = '/content/drive/My Drive/IE 534/Deep Dive Project - Energy Prices in Illinois/Data/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Load data

forecasted_load_df         = pd.read_csv(datadir + "20170823-20241020 MISO Forecasted Load.csv")
day_ahead_energy_price_df  = pd.read_csv(datadir + "20140101-20241020 MISO Day-Ahead Energy Price.csv")
actual_energy_price_df     = pd.read_csv(datadir + "20060410-20241020 MISO Actual Energy Price.csv")

Note: Locational Marginal Pricing (LMP) https://fresh-energy.org/negative-prices-in-the-miso-market-whats-happening-and-why-should-we-care

In [ ]:
#Display info about data

print("Forecasted Data\n\n")
print(forecasted_load_df.info())
print(f"Describe\n{forecasted_load_df.describe()}")
forecasted_load_df.head()

Forecasted Data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57086 entries, 0 to 57085
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    57086 non-null  object
 1   Load    57086 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 892.1+ KB
None
Describe
                Load
count   57086.000000
mean    76181.885909
std     11718.020694
min     50538.000000
25%     67975.000000
50%     74352.500000
75%     82270.750000
max    127195.000000


,Date,Load
0,8/23/2017 1:00:00 AM,71745
1,8/23/2017 2:00:00 AM,68985
2,8/23/2017 3:00:00 AM,67338
3,8/23/2017 4:00:00 AM,66788
4,8/23/2017 5:00:00 AM,67886


In [ ]:
#Display info about data

print("Day-ahead energy price\n\n")
print(day_ahead_energy_price_df.info())
print(f"Describe\n{day_ahead_energy_price_df.describe()}")
day_ahead_energy_price_df.head()

Day-ahead energy price


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723120 entries, 0 to 723119
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    723120 non-null  object 
 1   lmp     723120 non-null  float64
 2   node    723120 non-null  object 
dtypes: float64(1), object(2)
memory usage: 16.6+ MB
None
Describe
                 lmp
count  723120.000000
mean       32.569990
std        21.891411
min       -40.530000
25%        21.570000
50%        26.970000
75%        36.760000
max      1743.890000


,Date,lmp,node
0,1/1/2014 12:00:00 AM,34.43,ARKANSAS.HUB
1,1/1/2014 12:00:00 AM,26.63,ILLINOIS.HUB
2,1/1/2014 12:00:00 AM,27.78,INDIANA.HUB
3,1/1/2014 12:00:00 AM,32.28,LOUISIANA.HUB
4,1/1/2014 12:00:00 AM,28.69,MICHIGAN.HUB


In [ ]:
#Display info about data

print("Actual energy price\n\n")
print(actual_energy_price_df.info())
print(f"Describe\n{actual_energy_price_df.describe()}")
actual_energy_price_df.head()

Actual energy price


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828171 entries, 0 to 828170
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    828171 non-null  object 
 1   LMP     828171 non-null  float64
 2   HUB     828171 non-null  object 
dtypes: float64(1), object(2)
memory usage: 19.0+ MB
None
Describe
                 LMP
count  828171.000000
mean       32.694563
std       399.329255
min    -78647.700000
25%        21.300000
50%        26.350000
75%        37.270000
max      8362.070000


,Date,LMP,HUB
0,4/10/2006 11:00:00 AM,68.84,CINERGY.HUB
1,4/10/2006 11:00:00 AM,70.07,FE.HUB
2,4/10/2006 11:00:00 AM,68.04,ILLINOIS.HUB
3,4/10/2006 11:00:00 AM,73.33,MICHIGAN.HUB
4,4/10/2006 11:00:00 AM,63.43,MINN.HUB


Clean the data:
-Remove data from other hubs(CINERGY.HUB, FE.HUB)

In [ ]:
#Clean data

#- Convert to Pandas date/time
forecasted_load_df['Date']        = pd.to_datetime(forecasted_load_df['Date'])
day_ahead_energy_price_df['Date'] = pd.to_datetime(day_ahead_energy_price_df['Date'])
actual_energy_price_df['Date']    = pd.to_datetime(actual_energy_price_df['Date'])

# -Remove data from other hubs(ex. CINERGY.HUB, FE.HUB,ARKANSAS.HUB) from actual energy price
forecast_load_MISO           = forecasted_load_df
day_ahead_price_IL        = day_ahead_energy_price_df[day_ahead_energy_price_df['node'] == 'ILLINOIS.HUB']
actual_energy_price_IL     = actual_energy_price_df[actual_energy_price_df['HUB'] == 'ILLINOIS.HUB']
actual_energy_price_others     = actual_energy_price_df[actual_energy_price_df['HUB'] != 'ILLINOIS.HUB']
hubs = actual_energy_price_df[actual_energy_price_df['HUB'] != 'ILLINOIS.HUB']['HUB'].unique().tolist()
actual_energy_price_hubs = {hub:actual_energy_price_df[actual_energy_price_df['HUB'] == hub] for hub in hubs}


# Change the feature names so the are distinct
forecast_load_MISO           = forecast_load_MISO.rename(columns={'Load': 'Forecasted Load (MISO)'})
day_ahead_price_IL         = day_ahead_price_IL.rename(columns={'lmp': 'Day Ahead Energy Price (IL)'})
day_ahead_price_IL         = day_ahead_price_IL.rename(columns={'node': 'HUB'})
actual_energy_price_IL     = actual_energy_price_IL.rename(columns={'LMP': 'Energy Price (IL)'})
actual_energy_price_others     = actual_energy_price_others.rename(columns={'LMP': 'Energy Price (others)'})
for hub in actual_energy_price_hubs:
  actual_energy_price_hubs[hub] = actual_energy_price_hubs[hub].rename(columns={'LMP': f'Energy Price ({hub})'})

# Drop HUB names since it is now a separate feature
day_ahead_price_IL     = day_ahead_price_IL.drop(columns=['HUB'])
actual_energy_price_IL = actual_energy_price_IL.drop(columns=['HUB'])
for hub in actual_energy_price_hubs:
  actual_energy_price_hubs[hub] = actual_energy_price_hubs[hub].drop(columns=['HUB'])

<ipython-input-33-3b49a6cddf18>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  day_ahead_energy_price_df['Date'] = pd.to_datetime(day_ahead_energy_price_df['Date'])


In [ ]:
# Display cleaned data
print(forecast_load_MISO.head())
print(day_ahead_price_IL.head())
print(actual_energy_price_IL.head())
print(actual_energy_price_others.head())
for hub in actual_energy_price_hubs:
  print(actual_energy_price_hubs[hub].head())

                 Date  Forecasted Load (MISO)
0 2017-08-23 01:00:00                   71745
1 2017-08-23 02:00:00                   68985
2 2017-08-23 03:00:00                   67338
3 2017-08-23 04:00:00                   66788
4 2017-08-23 05:00:00                   67886
                  Date  Day Ahead Energy Price (IL)
1  2014-01-01 00:00:00                        26.63
8  2014-01-01 01:00:00                        25.87
15 2014-01-01 02:00:00                        25.41
22 2014-01-01 03:00:00                        25.21
29 2014-01-01 04:00:00                        24.62
                  Date  Energy Price (IL)
2  2006-04-10 11:00:00              68.04
7  2006-04-10 12:00:00              41.25
12 2006-04-10 13:00:00              35.40
17 2006-04-10 14:00:00              29.75
22 2006-04-10 15:00:00              39.05
                 Date  Energy Price (others)           HUB
0 2006-04-10 11:00:00                  68.84   CINERGY.HUB
1 2006-04-10 11:00:00                  70.

In [ ]:

merged_df = pd.merge(forecast_load_MISO, day_ahead_price_IL, on='Date', how='outer')
merged_df = pd.merge(merged_df, actual_energy_price_IL, on='Date', how='outer')
merged_df = pd.merge(merged_df, actual_energy_price_others, on='Date', how='outer')

# merged_area = list(actual_energy_price_hubs.values())[0]
# for hub in list(actual_energy_price_hubs.values())[1:7]:
#   merged_area = pd.merge(merged_area, hub, on='Date', how='outer')

# Sort the merged DataFrame by datetime
merged_df = merged_df.sort_values('Date')

merged_df

,Date,Forecasted Load (MISO),Day Ahead Energy Price (IL),Energy Price (IL),Energy Price (others),HUB
0,2006-04-10 11:00:00,NaN,NaN,68.04,68.84,CINERGY.HUB
1,2006-04-10 11:00:00,NaN,NaN,68.04,70.07,FE.HUB
2,2006-04-10 11:00:00,NaN,NaN,68.04,73.33,MICHIGAN.HUB
3,2006-04-10 11:00:00,NaN,NaN,68.04,63.43,MINN.HUB
4,2006-04-10 12:00:00,NaN,NaN,41.25,41.62,CINERGY.HUB
...,...,...,...,...,...,...
723835,2024-10-20 19:00:00,NaN,24.33,NaN,NaN,NaN
723836,2024-10-20 20:00:00,NaN,21.48,NaN,NaN,NaN
723837,2024-10-20 21:00:00,NaN,17.90,NaN,NaN,NaN
723838,2024-10-20 22:00:00,NaN,15.20,NaN,NaN,NaN


In [ ]:

cleaned_df = merged_df.dropna()
print(cleaned_df)

                      Date  Forecasted Load (MISO)  \
396395 2017-08-24 00:00:00                 73102.0   
396393 2017-08-24 00:00:00                 73102.0   
396394 2017-08-24 00:00:00                 73102.0   
396391 2017-08-24 00:00:00                 73102.0   
396390 2017-08-24 00:00:00                 73102.0   
...                    ...                     ...   
723826 2024-10-20 14:00:00                 65389.0   
723827 2024-10-20 14:00:00                 65389.0   
723828 2024-10-20 14:00:00                 65389.0   
723829 2024-10-20 14:00:00                 65389.0   
723830 2024-10-20 14:00:00                 65389.0   

        Day Ahead Energy Price (IL)  Energy Price (IL)  Energy Price (others)  \
396395                        18.06              19.31                  20.79   
396393                        18.06              19.31                  20.37   
396394                        18.06              19.31                  17.67   
396391                     

In [ ]:
# Create the Month, day of week, and hour features
# use Cyclical Data. Source: https://betterdatascience.com/cyclical-data-machine-learning/

merged_df['Month']       = merged_df['Date'].dt.month
merged_df['Day of week'] = merged_df['Date'].dt.dayofweek
merged_df['Hour']        = merged_df['Date'].dt.hour

#convert to cyclical data
merged_df['Sin_Month'] = np.sin(2 * np.pi * merged_df['Month'] / max(merged_df['Month']))
merged_df['Cos_Month'] = np.cos(2 * np.pi * merged_df['Month'] / max(merged_df['Month']))

merged_df['Sin_Day'] = np.sin(2 * np.pi * merged_df['Day of week'] / max(merged_df['Day of week']))
merged_df['Cos_Day'] = np.cos(2 * np.pi * merged_df['Day of week'] / max(merged_df['Day of week']))

merged_df['Sin_Hour'] = np.sin(2 * np.pi * merged_df['Hour'] / max(merged_df['Hour']))
merged_df['Cos_Hour'] = np.cos(2 * np.pi * merged_df['Hour'] / max(merged_df['Hour']))

# delete the day, month, hour since we have the sin/cos
merged_df.drop(columns=['Month'], inplace=True)
merged_df.drop(columns=['Day of week'], inplace=True)
merged_df.drop(columns=['Hour'], inplace=True)


merged_df.head(40)

,Date,Forecasted Load (MISO),Day Ahead Energy Price (IL),Energy Price (IL),Energy Price (others),HUB,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Sin_Hour,Cos_Hour
0,2006-04-10 11:00:00,NaN,NaN,68.04,68.84,CINERGY.HUB,0.866025,-0.5,0.0,1.0,0.136167,-0.990686
1,2006-04-10 11:00:00,NaN,NaN,68.04,70.07,FE.HUB,0.866025,-0.5,0.0,1.0,0.136167,-0.990686
2,2006-04-10 11:00:00,NaN,NaN,68.04,73.33,MICHIGAN.HUB,0.866025,-0.5,0.0,1.0,0.136167,-0.990686
3,2006-04-10 11:00:00,NaN,NaN,68.04,63.43,MINN.HUB,0.866025,-0.5,0.0,1.0,0.136167,-0.990686
4,2006-04-10 12:00:00,NaN,NaN,41.25,41.62,CINERGY.HUB,0.866025,-0.5,0.0,1.0,-0.136167,-0.990686
5,2006-04-10 12:00:00,NaN,NaN,41.25,41.98,FE.HUB,0.866025,-0.5,0.0,1.0,-0.136167,-0.990686
6,2006-04-10 12:00:00,NaN,NaN,41.25,44.37,MICHIGAN.HUB,0.866025,-0.5,0.0,1.0,-0.136167,-0.990686
7,2006-04-10 12:00:00,NaN,NaN,41.25,38.44,MINN.HUB,0.866025,-0.5,0.0,1.0,-0.136167,-0.990686
11,2006-04-10 13:00:00,NaN,NaN,35.40,32.82,MINN.HUB,0.866025,-0.5,0.0,1.0,-0.398401,-0.917211
10,2006-04-10 13:00:00,NaN,NaN,35.40,37.99,MICHIGAN.HUB,0.866025,-0.5,0.0,1.0,-0.398401,-0.917211


In [ ]:
# Keep only the following HUBs: MICHIGAN.HUB, MINN.HUB, INDIANA.HUB, TEXAS.HUB, LOUISIANA.HUB, ARKANSAS.HUB, MS.HUB
# We only want these HUBs because we want data from 2017 to 2024 because we don't have forcasted load until 2017

# Filter out data before 2017
merged_df = merged_df[merged_df['Date'] >= '2017-01-01']

cleaned_df = merged_df.drop(columns=['Energy Price (others)', 'HUB'])
print(cleaned_df.head())

hubs = ['MICHIGAN.HUB', 'MINN.HUB', 'INDIANA.HUB', 'TEXAS.HUB', 'LOUISIANA.HUB', 'ARKANSAS.HUB', 'MS.HUB']
for hub in hubs:
  hub_data = merged_df[merged_df['HUB'] == hub]
  hub_data = hub_data[['Date','Energy Price (others)']]
  hub_data = hub_data.rename(columns={"Energy Price (others)": f"Energy Price ({hub})"}).drop_duplicates().reset_index(drop=True)
  cleaned_df = cleaned_df.merge(hub_data, how='left', on='Date')

cleaned_df.head()

print(f"Shape before cleaning: {merged_df.shape}, shape after cleaning: {cleaned_df.shape}")

             Date  Forecasted Load (MISO)  Day Ahead Energy Price (IL)  \
364129 2017-01-01                     NaN                        23.73   
364128 2017-01-01                     NaN                        23.73   
364130 2017-01-01                     NaN                        23.73   
364126 2017-01-01                     NaN                        23.73   
364125 2017-01-01                     NaN                        23.73   

        Energy Price (IL)  Sin_Month  Cos_Month       Sin_Day  Cos_Day  \
364129              29.53        0.5   0.866025 -2.449294e-16      1.0   
364128              29.53        0.5   0.866025 -2.449294e-16      1.0   
364130              29.53        0.5   0.866025 -2.449294e-16      1.0   
364126              29.53        0.5   0.866025 -2.449294e-16      1.0   
364125              29.53        0.5   0.866025 -2.449294e-16      1.0   

        Sin_Hour  Cos_Hour  
364129       0.0       1.0  
364128       0.0       1.0  
364130       0.0       

In [ ]:
#Temp data
# Load the CSV file
df=pd.read_csv(datadir + 'Air_temperature_argonne.txt', sep="\t", dtype={0: int, 1: str, 2: str, 3: str, 4:float, 5:int})
# Filter rows where Year is greater than 2021
df = df[df['Year'] >= 2017]
df.reset_index(drop=True, inplace=True)
df['Year'] = df['Year'].astype(str)
df.columns = ['Year', 'Month',	'Day',	'Hour',	'Temperature','Flag']
# Modify column values based on their length
df['Hour'] = df['Hour'].apply(lambda x: '0' + x[0] if len(x) != 4 else x[0:2])
df['Hour'] = df['Hour'].astype(str)
df['Hour'] = df['Hour'].apply(lambda x: '00' if x == '24' else x)
df['Hour'] = df['Hour'].apply(lambda x: x + ':00:00')
df.head()

,Year,Month,Day,Hour,Temperature,Flag
0,2017,01,01,01:00:00,23.00,110
1,2017,01,01,02:00:00,22.10,0
2,2017,01,01,03:00:00,21.38,0
3,2017,01,01,04:00:00,20.66,0
4,2017,01,01,05:00:00,19.94,0


In [ ]:
# Create a new DataFrame with the specified format
df['Date'] = df[['Year', 'Month', 'Day']].astype(str).agg('-'.join, axis=1)
df['Timestamp'] = df[['Date', 'Hour']].astype(str).agg(' '.join, axis=1)
df['Date'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')
temperature = df[['Date', 'Temperature']]
temperature.head(25)

,Date,Temperature
0,2017-01-01 01:00:00,23.00
1,2017-01-01 02:00:00,22.10
2,2017-01-01 03:00:00,21.38
3,2017-01-01 04:00:00,20.66
4,2017-01-01 05:00:00,19.94
5,2017-01-01 06:00:00,19.58
6,2017-01-01 07:00:00,19.58
7,2017-01-01 08:00:00,20.66
8,2017-01-01 09:00:00,23.36
9,2017-01-01 10:00:00,27.86


In [ ]:
# Merge temp
cleaned_df = cleaned_df.merge(temperature, how='left', on='Date')
cleaned_df.head()

,Date,Forecasted Load (MISO),Day Ahead Energy Price (IL),Energy Price (IL),Sin_Month,Cos_Month,Sin_Day,Cos_Day,Sin_Hour,Cos_Hour,Energy Price (MICHIGAN.HUB),Energy Price (MINN.HUB),Energy Price (INDIANA.HUB),Energy Price (TEXAS.HUB),Energy Price (LOUISIANA.HUB),Energy Price (ARKANSAS.HUB),Energy Price (MS.HUB),Temperature
0,2017-01-01,NaN,23.73,29.53,0.5,0.866025,-2.449294e-16,1.0,0.0,1.0,23.37,13.34,23.19,22.84,23.81,22.28,NaN,31.46
1,2017-01-01,NaN,23.73,29.53,0.5,0.866025,-2.449294e-16,1.0,0.0,1.0,23.37,13.34,23.19,22.84,23.81,22.28,NaN,31.46
2,2017-01-01,NaN,23.73,29.53,0.5,0.866025,-2.449294e-16,1.0,0.0,1.0,23.37,13.34,23.19,22.84,23.81,22.28,NaN,31.46
3,2017-01-01,NaN,23.73,29.53,0.5,0.866025,-2.449294e-16,1.0,0.0,1.0,23.37,13.34,23.19,22.84,23.81,22.28,NaN,31.46
4,2017-01-01,NaN,23.73,29.53,0.5,0.866025,-2.449294e-16,1.0,0.0,1.0,23.37,13.34,23.19,22.84,23.81,22.28,NaN,31.46


In [ ]:
print(cleaned_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359715 entries, 0 to 359714
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Date                          359715 non-null  datetime64[ns]
 1   Forecasted Load (MISO)        321699 non-null  float64       
 2   Day Ahead Energy Price (IL)   359553 non-null  float64       
 3   Energy Price (IL)             341160 non-null  float64       
 4   Sin_Month                     359715 non-null  float64       
 5   Cos_Month                     359715 non-null  float64       
 6   Sin_Day                       359715 non-null  float64       
 7   Cos_Day                       359715 non-null  float64       
 8   Sin_Hour                      359715 non-null  float64       
 9   Cos_Hour                      359715 non-null  float64       
 10  Energy Price (MICHIGAN.HUB)   341160 non-null  float64       
 11  Energy Price 

In [ ]:
# We can see that MS.HUB has about 31,000 less non-NaN samples than the lowest, we could drop MS.HUB
# But we will just drop all NaN
cleaned_df = cleaned_df.dropna()
cleaned_df.reset_index(drop=True, inplace=True)
print(cleaned_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228830 entries, 0 to 228829
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Date                          228830 non-null  datetime64[ns]
 1   Forecasted Load (MISO)        228830 non-null  float64       
 2   Day Ahead Energy Price (IL)   228830 non-null  float64       
 3   Energy Price (IL)             228830 non-null  float64       
 4   Sin_Month                     228830 non-null  float64       
 5   Cos_Month                     228830 non-null  float64       
 6   Sin_Day                       228830 non-null  float64       
 7   Cos_Day                       228830 non-null  float64       
 8   Sin_Hour                      228830 non-null  float64       
 9   Cos_Hour                      228830 non-null  float64       
 10  Energy Price (MICHIGAN.HUB)   228830 non-null  float64       
 11  Energy Price 

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Separate the date feature
date_feature = cleaned_df['Date']

# Select the features to be normalized
features_to_normalize = cleaned_df.drop(columns=['Date'])

# Initialize the StandardScaler
norm = MinMaxScaler()

# Fit and transform the data
normalized_features = pd.DataFrame(norm.fit_transform(features_to_normalize), columns=features_to_normalize.columns)

# Combine the normalized features with the date feature
cleaned_df = pd.concat([date_feature, normalized_features], axis=1)

print(cleaned_df.head())


                 Date  Forecasted Load (MISO)  Day Ahead Energy Price (IL)  \
0 2017-12-04 12:00:00                0.332677                     0.076424   
1 2017-12-04 12:00:00                0.332677                     0.076424   
2 2017-12-04 12:00:00                0.332677                     0.076424   
3 2017-12-04 12:00:00                0.332677                     0.076424   
4 2017-12-04 12:00:00                0.332677                     0.076424   

   Energy Price (IL)  Sin_Month  Cos_Month  Sin_Day  Cos_Day  Sin_Hour  \
0           0.042684        0.5        1.0      0.5      1.0  0.431758   
1           0.042684        0.5        1.0      0.5      1.0  0.431758   
2           0.042684        0.5        1.0      0.5      1.0  0.431758   
3           0.042684        0.5        1.0      0.5      1.0  0.431758   
4           0.042684        0.5        1.0      0.5      1.0  0.431758   

   Cos_Hour  Energy Price (MICHIGAN.HUB)  Energy Price (MINN.HUB)  \
0       0.0      

In [ ]:
debugging = cleaned_df.sample(n=1000, random_state=42)
working = cleaned_df.sample(n=100000, random_state=42)
everything = cleaned_df

print(debugging.head())
print(working.head())

                      Date  Forecasted Load (MISO)  \
216436 2023-07-15 01:00:00                0.374186   
83195  2020-09-03 15:00:00                0.563458   
22395  2018-08-27 16:00:00                0.859126   
119149 2021-10-04 21:00:00                0.370860   
120945 2021-10-24 21:00:00                0.263681   

        Day Ahead Energy Price (IL)  Energy Price (IL)  Sin_Month  Cos_Month  \
216436                     0.074279           0.042462   0.250000   0.066987   
83195                      0.107889           0.043390   0.000000   0.500000   
22395                      0.172272           0.072441   0.066987   0.250000   
119149                     0.165836           0.061022   0.066987   0.750000   
120945                     0.126205           0.049354   0.066987   0.750000   

        Sin_Day  Cos_Day  Sin_Hour  Cos_Hour  Energy Price (MICHIGAN.HUB)  \
216436      0.0     0.75  0.635214  0.981372                     0.022101   
83195       0.5     0.00  0.090561  0.20

In [ ]:

debugging.to_pickle('./debugging.pkl')
working.to_pickle('./working.pkl')
everything.to_pickle('./all_data.pkl')
